In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Wind.Results import DBResults
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import seaborn as sns
from Wind.Private.DBConfig import mongolocaltest, mongolocal
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff

# Fitting RNN Encoder Decoder

## Fit1

First fit experiment RNN seq2seq, explore dropout and number of input and output neurons

In [132]:

query={'experiment':'rnnfit'}

results = DBResults(conn=mongolocaltest)
#results = DBResults()

results.results_dataframe_retrieve(query, arch=['neuronsD', 'nlayersD', 'neuronsE', 'nlayersE', 'drop','activation'], data=['lag'])


In [135]:
results.results_dataframe_aggregate(threshold=6.81)

In [138]:
results.results_dataframe_graph(facet='neuronsE',att=['neuronsD','neuronsE','nlayersD','nlayersE', 'drop'])

Best results  96/160/0.0

Second fit of RNN, explore type of unit, lag and some more neuronsE and neuronsD

## Fit2

In [126]:
query={'experiment':'rnnfit2'}

results = DBResults(conn=mongolocaltest)
#results = DBResults()
results.results_dataframe_retrieve(query, arch=['neuronsD', 'nlayersD', 'neuronsE', 'nlayersE', 'drop','activation', 'rnn'], data=['lag'])


In [129]:
results.results_dataframe_aggregate(threshold=6.82)

In [131]:
results.results_dataframe_graph(facet='rnn',att=['neuronsD','neuronsE','nlayersD','nlayersE', 'drop', 'rnn'])

Best results 196/128/0.0/12/GRU

## Fit3

Third fit of RNN, explore number of layers

In [120]:
query={'experiment':'rnnfit3'}

results = DBResults(conn=mongolocaltest)
#results = DBResults()

results.results_dataframe_retrieve(query, arch=['neuronsD', 'nlayersD', 'neuronsE', 'nlayersE', 'drop','activation'], data=['lag'])

In [123]:
results.results_dataframe_aggregate(threshold=6.84)

In [125]:
results.results_dataframe_graph(facet='neuronsD',att=['neuronsD','neuronsE','nlayersD','nlayersE', 'drop'])

Best results 196(1)/128(2)/0.0/GRU/lag=12

## Fit 4

Fourth fit of RNN, reexplore lag, decoder neurons, dropout and decoder layers

In [116]:
query={'experiment':'rnnfit4'}

results = DBResults(conn=mongolocaltest)
#results = DBResults()

results.results_dataframe_retrieve(query, arch=['neuronsD', 'nlayersD', 'neuronsE', 'nlayersE', 'drop','activation'], data=['lag'])

In [117]:
results.results_dataframe_aggregate(threshold=6.85)

7200


In [118]:
results.results_dataframe_graph(facet='neuronsD',att=['neuronsD','nlayersD', 'drop'])

final best configuration GRU/196/1/196/1/0.0/18

## Fit 5

Fifth fit of RNN, reexplore lag, decoder neurons, dropout and decoder layers

In [109]:
query={'experiment':'rnnedfit5'}

results = DBResults(conn=mongolocaltest)
#results = DBResults()

results.results_dataframe_retrieve(query, arch=['neuronsD', 'nlayersD', 'neuronsE', 'nlayersE', 'drop', 'full', 'fulldrop','activation','activation_full'], data=['lag'])

In [110]:
results.results_dataframe_aggregate(threshold=6.81)

5600


In [111]:
results.results_dataframe_graph(facet='activation_full',att=['neuronsE','neuronsD','nlayersE','nlayersD', 'fulldrop', 'drop','activation', 'activation_full','lag'])

final best configuration 

# Bayesian Fitting

In [3]:
query={'experiment':'rnns2sedbayesian'}

results = DBResults(conn=mongolocaltest)
#results = DBResults()

results.results_dataframe_retrieve(query, arch=['neuronsE', 'neuronsD','nlayersE','nlayersD','activation', 'drop', 'full', 'fulldrop','activation_full'], data=['lag'])

In [4]:
results.results_dataframe_aggregate(threshold=6.85)

33352


In [5]:
results.results_dataframe_graph(facet='activation_full',att=['neuronsE','neuronsD','nlayersE','nlayersD', 'activation','fulldrop', 'drop', 'activation_full','lag'])